In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from qutip import *

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In this notebook, we show some examples of state-to-state transitions driven by $\pi$-pulses. The goal is to show how this method to achieve state-to-state transitions requires low pulse amplitudes, or else the quality of the transition fidelity is reduced. Therefore, the transition times must also be long, since they are inversely proportional to the pulse amplitudes.

The use of complex pulses found with optimal control theory permit, in theory, to accelerate the pulses: use shorter pulses that however have good fidelity. However, it is important to know the range of amplitudes that we can play with: the value, in Teslas, of the time-dependent magnetic field that is applied through the transmission lines. In the following, we have assumed maximum values of 1 mT.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

We consider the basis of eigenstates of $\hat{H}_0$:
\begin{equation}
\hat{H}_0\vert\psi_i\rangle = \varepsilon_i \vert\psi_i\rangle
\end{equation}
The characteristic frequencies are the energy differences:
\begin{equation}
\omega_i = \varepsilon_{i+1}-\varepsilon_{i}.
\end{equation}
(we only consider the frequencies between neighbour energies).

We will also need the coupling between the neighbour eigenstates:
\begin{equation}
\mu_{i,i+1} = \langle\psi_i\vert\hat{V}\vert\psi_{i+1}\rangle\,.
\end{equation}

The following describes the use of $\pi$-pulses sequences to drive the
same GdW$_{30}$ system described in the previous reports from the
ground state $\vert 0\rangle$ to both the linear combinations
$\frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 2 \rangle)$ and
$\frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 7 \rangle)$.  The system
description and parameters are therefore the same as in the previous
cases.  The idea now is to apply a $\pi/2$ pulse with $\omega_1$
frequency to build first a linear combination
$\frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 1 \rangle)$, and then
either (1) one $\pi$-pulse with frequency $\omega_2$ to drive then the
system to the $\frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 2 \rangle)$,
or (2) a succession of $\pi$-pulses with frequencies
$\omega_2,\dots,\omega_7$ to consecutively populate each state of the
system, and finally take the system to the $\frac{1}{\sqrt{2}}(\vert 0
\rangle + \vert 7 \rangle)$ state.

Since we do not have two-level systems, but there are eight levels
present, the $\pi$-pulses do not create exact transitions, but there
is always some contamination with other levels, that can be avoided if
there are no resonances, and the interaction is weak. Therefore, I
have searched for the regime of intensities of the td microwave pulse
that can be used, so that we get an idea of the time scales.

In [ ]:
# Parameters definition
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

n = 7 # max level of the final state

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates)
V = V.transform(eigenstates)

# $\pi$-pulses

We define the following "simple" $\pi$-pulses:
\begin{equation}
g_{[i,i+1,t_0,\pi]}(t) = 
\begin{cases}
A \sin(\omega_{i,i+1}t) & t_0 \le t \le t_f = t_0 + \frac{\pi}{A\vert\mu_{i,i+1}\vert}
\\
0 & \textrm{otherwise}
\end{cases}
\end{equation}
If the system is at $\psi_i$ at time $t_0$, then this pulse produces an \emph{approximate} transition
to state $\psi_{i+1}$. It will become an almost exact transition in the limit in which $A$ approaches
zero (as long as there are no degenerate resonances). The time $t_0$ i the starting time of the pulse, whereas the time $t_f$ is the end point. The duration is inversely proportional to the amplitude.

We also consider $\pi/2$-pulses $g_{[i,i+1,t_0,\pi/2]}$, that produce approximate transitions from $\psi_i$ to $\frac{1}{\sqrt{2}}(\psi_i+\psi_{i+1})$. They are equal, except they have half the duration.

For example, for the transition $\psi_0 \to \frac{1}{\sqrt{2}}(\psi_0 + \psi_7)$, we build a sequence of $\pi$-pulses that take the system from state to state (since the direct coupling is zero):
\begin{eqnarray}
\nonumber 0 &:&  g_{[0,1,t_0^0=0,\pi/2]}
\\
\nonumber 1 &:&  g_{[1,2,t_0^1 = t_f^0,\pi]}
\\
\nonumber 2 &:&  g_{[2,3,t_0^2 = t_f^1,\pi]}
\\
\nonumber 3 &:&  g_{[3,4,t_0^3 = t_f^2,\pi]}
\\
\nonumber 4 &:&  g_{[4,5,t_0^4 = t_f^3,\pi]}
\\
\nonumber 5 &:&  g_{[5,6,t_0^5 = t_f^4,\pi]}
\\
\nonumber 6 &:&  g_{[6,7,t_0^6 = t_f^5,\pi]}
\end{eqnarray}
The times $t_0^i,t_f^i$ are the initial and final times of the $i$-th pulse. In words, a $\pi/2$ pulse that takes the system from $\psi_0$ to $\frac{1}{\sqrt{2}}(\psi_0+\psi_1)$, then a $\pi$ pulse that takes the pulse from $\frac{1}{\sqrt{2}}(\psi_0+\psi_1)$ to $\frac{1}{\sqrt{2}}(\psi_0+\psi_2)$, $\dots$, and finally a $\pi$ pulse that takes the pulse from $\frac{1}{\sqrt{2}}(\psi_0+\psi_6)$ to $\frac{1}{\sqrt{2}}(\psi_0+\psi_7)$.

For the transition $\psi_0 \to \frac{1}{\sqrt{2}}(\psi_0 + \psi_2)$, we only need two pulses instead of seven.

In [ ]:
w = []
for i in range(int(dim - 1)):
    w.append(eigenvalues[i+1] - eigenvalues[i]) #in MHz

# We will define a "characteristic period" of the field-free Hamiltonian
# as the period corresponding to the maximum frequency. Note that we are
# considering only the "neighbour energies" to define the frequencies.
tau0 = (2*np.pi/np.max(np.array(w)))

## Large amplitude, fast process, low fidelity

First, we will consider an amplitude $A$ of 1 mT. The results can be seen in the following figure: it can be seen how the transition from state to state is not perfect, and as a consequence the population of $\psi_7$ at the end of the process is very low.

In [ ]:
A = 1.0
n = 7

In [ ]:
pi_pulse_duration = []
for i in range(dim - 1):
    pi_pulse_duration.append(np.pi/(A*abs(V.data[i, i+1])))

u = np.zeros(2+2*n)
u[0] = A
u[1] = n
for i in range(n):
    u[2+i] = w[i]
    u[2+n+i] = pi_pulse_duration[i]
u[2+n] = u[2+n]/2.0

In [ ]:
max_time = 0
for i in range(n):
    max_time += u[2+n+i]

In [ ]:
f = pulses.pulse('user_defined', max_time, u = u)
f.assign_user_defined_function(pulses.pi_pulse_chain, None)

In [ ]:
# We will define the time step by considering the characteristic period.
nperiods = max_time / tau0
ntimestepsperperiod = 2
dt = max_time / (int(nperiods)*ntimestepsperperiod)
time = np.linspace(0.0, max_time, int(nperiods*ntimestepsperperiod))

In [ ]:
# Initial condition (starting from the ground state)
psi0 = basis(dim, 0) #initial state

In [ ]:
result = solvers.solve('sesolve', hamiltonians.hamiltonian(H0, V), f, psi0, time,
                       returnQoutput = True)

In [ ]:
x = np.zeros((time.size,n + 2))
x[:, 0] = np.array(time)
for i in range(time.size): #lopp over each time value
    for j in range(n + 1): #loop over each energy level
        x[i, j + 1] = abs(result.states[i].data[j, 0])**2

plt.xlabel("time(us)")
plt.ylabel("ocupation")
plt.title("|0> to 0.707*(|0> + |{}>) transition with pulsos-pi".format(n))
plt.xlim(0, max_time/(2.0*np.pi))
plt.ylim(0,1)
plt.grid("best")
for i in range(n + 1):
    plt.plot(x[:, 0]/(2.0*np.pi), x[:, i + 1], label = "|{}>".format(i))
plt.legend(loc='upper center', ncol=2, fancybox=True, shadow=True)
if isnotebook:
    plt.show()

In [ ]:
data.append(x[-1, -1])

## Smaller amplitude, slower process, high fidelity

Then, an amplitude of 0.1 mT. In this case, the result is almost perfect. The process must be slow (since the $\pi$-pulse duration is inversely proportional to the amplitude), but each transition leads the system to the proper states, and finally the population of both the $\psi_0$ and the $\psi_7$ state is almost 1/2.

In [ ]:
A = 0.1

In [ ]:
pi_pulse_duration = []
for i in range(dim - 1):
    pi_pulse_duration.append(np.pi/(A*abs(V.data[i, i+1])))

u = np.zeros(2+2*n)
u[0] = A
u[1] = n
for i in range(n):
    u[2+i] = w[i]
    u[2+n+i] = pi_pulse_duration[i]
u[2+n] = u[2+n]/2.0
f.set_parameters(u)

In [ ]:
max_time = 0
for i in range(n):
    max_time += u[2+n+i]

In [ ]:
# We will define the time step by considering the characteristic period.
nperiods = max_time / tau0
ntimestepsperperiod = 2
dt = max_time / (int(nperiods)*ntimestepsperperiod)
time = np.linspace(0.0, max_time, int(nperiods*ntimestepsperperiod))

In [ ]:
psi0 = basis(dim, 0) #initial state

In [ ]:
result = solvers.solve('sesolve', hamiltonians.hamiltonian(H0, V), f, psi0, time,
                       returnQoutput = True)

In [ ]:
x = np.zeros((time.size,n + 2))
x[:, 0] = np.array(time)
for i in range(time.size): #lopp over each time value
    for j in range(n + 1): #loop over each energy level
        x[i, j + 1] = abs(result.states[i].data[j, 0])**2

plt.xlabel("time(us)")
plt.ylabel("ocupation")
plt.title("|0> to 0.707*(|0> + |{}>) transition with pulsos-pi".format(n))
plt.xlim(0, max_time/(2.0*np.pi))
plt.ylim(0,1)
plt.grid("best")
for i in range(n + 1):
    plt.plot(x[:, 0]/(2.0*np.pi), x[:, i + 1], label = "|{}>".format(i))
plt.legend(loc='upper center', ncol=2, fancybox=True, shadow=True)
if isnotebook:
    plt.show()

In [ ]:
data.append(x[-1, -1])

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))

# Conclusions 

These calculations give us an idea of the times needed to produce state-to-state
transitions with $\pi$-pulses, which are probably not too different to the times
needed to produce gates.

These times must be compared with the decoherence times of the system.